# The Guardian API

In the `beautiful_soup.ipynb` notebook, I showed how BeautifulSoup can be used 
to parse messy HTML, tp extract information, and to act as a rudimentary web crawler. 
I used The Guardian as an illustrative example about how this can be achieved. 
The reason for choosing The Guardian was because they provide a REST API to their servers. 
With theise it is possible to perform specific queries on their servers, and to receive 
current information from their servers according to their API guide (ie in JSON)

http://open-platform.theguardian.com/

In order to use their API, you will need to register for an API key. 
At the time of writing (Feb 1, 2017) this was an automated process that can be completed at 

http://open-platform.theguardian.com/access/

The API is documented here: 

http://open-platform.theguardian.com/documentation/

and Python bindings to their API are provided by The Guardian here

https://github.com/prabhath6/theguardian-api-python

and these can easily be integrated into a web-crawler based on API calls, rather than being based 
on HTML parsing, etc. 

We use four parameters in our queries here: 

1. `section`: the section of the newspaper that we are interested in querying. In this case I'm lookin in 
the technology section 

2. `order-by`: I have specifie that the newest items should be closer to the front of the query list 

3. `api-key`: I have left this as test (which works here), but for *real* deployment of such a spider
a real API key should be specified 

4. `page-size`: The number of results to return. 

In [2]:
from __future__ import print_function

import requests 
import json 

# Inspect all sections and search for technology-based sections

In [3]:
url = 'https://content.guardianapis.com/sections?api-key=test'
req = requests.get(url)
src = req.text 

In [12]:
json.loads(src)['response']['status']

'ok'

In [11]:
sections = json.loads(src)['response']

print(sections.keys())

dict_keys(['status', 'userTier', 'total', 'results'])


In [13]:
print(json.dumps(sections['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/about",
  "editions": [
    {
      "apiUrl": "https://content.guardianapis.com/about",
      "code": "default",
      "id": "about",
      "webTitle": "About",
      "webUrl": "https://www.theguardian.com/about"
    }
  ],
  "id": "about",
  "webTitle": "About",
  "webUrl": "https://www.theguardian.com/about"
}


In [14]:
for result in sections['results']: 
    if 'tech' in result['id'].lower(): 
        print(result['webTitle'], result['apiUrl'])

Technology https://content.guardianapis.com/technology


# Manual query on whole API

In [23]:
# Specify the arguments
args = {
    'section': 'technology', 
    'order-by': 'newest', 
    'api-key': 'test', 
    'page-size': '100',
    'q' : 'privacy%20AND%20data'
}

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

In [24]:
print('Number of byes received:', len(src))

Number of byes received: 59513


The API returns JSON, so we parse this using the in-built JSON library. 
The API specifies that all data are returned within the `response` key, even under failure. 
Thereofre, I have immediately descended to the response field 

# Parsing the JSON

In [25]:
response = json.loads(src)['response']
print('The following are available:\n ', sorted(response.keys()))

The following are available:
  ['currentPage', 'orderBy', 'pageSize', 'pages', 'results', 'startIndex', 'status', 'total', 'userTier']


# Verifying the status code

It is important to verify that the status message is `ok` before continuing - if it is not `ok` no 'real' data 
will have been received. 

In [26]:
assert response['status'] == 'ok'

# Listing the results 

The API standard states that the results will be found in the `results` field under the `response` field. 
Furthermore, the URLs will be found in the `webUrl` field, and the title will be found in the `webTitle` 
field. 

First let's look to see what a single result looks like in full, and then I will print a restricted 
set of parameters on the full set of results .

In [27]:
print(json.dumps(response['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/technology/2020/feb/04/google-software-glitch-sent-some-users-videos-to-strangers",
  "id": "technology/2020/feb/04/google-software-glitch-sent-some-users-videos-to-strangers",
  "isHosted": false,
  "pillarId": "pillar/news",
  "pillarName": "News",
  "sectionId": "technology",
  "sectionName": "Technology",
  "type": "article",
  "webPublicationDate": "2020-02-04T12:56:37Z",
  "webTitle": "Google software glitch sent some users' videos to strangers",
  "webUrl": "https://www.theguardian.com/technology/2020/feb/04/google-software-glitch-sent-some-users-videos-to-strangers"
}


In [46]:
for result in response['results']: 
    print(result['webUrl'][:70], result['webTitle'][:20])

https://www.theguardian.com/technology/2020/feb/04/google-software-gli Google software glit
https://www.theguardian.com/technology/2020/jan/30/mike-pompeo-restate Mike Pompeo restates
https://www.theguardian.com/technology/2020/jan/30/facebook-pays-550m- Facebook pays $550m 
https://www.theguardian.com/technology/2020/jan/28/boris-johnson-gets- Boris Johnson gets f
https://www.theguardian.com/technology/commentisfree/2020/jan/25/facia Quick, cheap to make
https://www.theguardian.com/technology/2020/jan/25/peter-diamandis-fut Peter Diamandis: ‘In
https://www.theguardian.com/technology/2020/jan/24/met-police-begin-us Met police to begin 
https://www.theguardian.com/technology/2020/jan/22/tell-us-about-the-w Tell us about the we
https://www.theguardian.com/technology/2020/jan/22/un-investigators-to Bezos hack: UN to ad
https://www.theguardian.com/technology/2020/jan/22/tech-firms-fail-pro Watchdog cracks down
https://www.theguardian.com/technology/2020/jan/18/1-trillion-dollars- $1tn is j

Let's now request a specific piece of content from the API. 

We select the ith result from the above response and get its ```apiUrl``` and ```id```:

In [50]:
i = 0
api_url = response['results'][i]['apiUrl']
api_id = response['results'][i]['id']

print(api_url)
print(api_id)

https://content.guardianapis.com/technology/2020/feb/04/google-software-glitch-sent-some-users-videos-to-strangers
technology/2020/feb/04/google-software-glitch-sent-some-users-videos-to-strangers


We then use the ```id``` to contstruct a search url string to request this piece of content from the API.

(Note that you need to include the ```api-key``` in the search, this is what I forgot in the lecture. You also need to specify if you want to include data fields other than the article metadata e.g. ```body``` and ```headline``` are included in the example below.)

In [53]:
base_url = "https://content.guardianapis.com/search?"
search_string = "ids=%s&api-key=test&show-fields=headline,body" %api_id

url = base_url + search_string
print(url)

https://content.guardianapis.com/search?ids=technology/2020/feb/04/google-software-glitch-sent-some-users-videos-to-strangers&api-key=test&show-fields=headline,body


In [54]:
req = requests.get(url) 
src = req.text

In [57]:
response = json.loads(src)['response']
assert response['status'] == 'ok'

In [60]:
print(response['results'][0]['fields']['headline'])

Google software glitch sent some users' videos to strangers


In [62]:
body = response['results'][0]['fields']['body']
print(body)

<p>Google has said a software bug resulted in some users’ personal videos being emailed to strangers.</p> <p>The flaw affected users of Google Photos who requested to export their data in late November. For four days the export tool wrongly added videos to unrelated users’ archives.</p> <p>As a result, private videos may have been sent to strangers, while downloaded archives may not have been complete.</p> <p>“We are notifying people about a bug that may have affected users who used Google Takeout to export their Google Photos content between November 21 and November 25,” a Google spokesperson said.</p> <p>“These users may have received either an incomplete archive, or videos – not photos – that were not theirs. We fixed the underlying issue and have conducted an in-depth analysis to help prevent this from ever happening again. We are very sorry this happened.”</p> <p>The company emphasised that the bug only affected users of Google Takeout, a download-your-data tool, and not users of 

We can now do some simple text processing on the article text. e.g. count the word frequnecies:

In [94]:
words = body.replace('<p>','').replace('</p>','').split()
print(len(words))
unique_words = list(set(words))
print(len(unique_words))
#count_dictionary = {word: count for word, count in zip(words, [words.count(w) for w in words])}
count_dictionary = {'word': unique_words, 'count': [words.count(w) for w in unique_words]}

422
240


In [96]:
import pandas as pd

In [97]:
df = pd.DataFrame(count_dictionary)
df.sort_values(by='count', ascending=False)

,word,count
137,the,16
65,to,14
23,Google,14
5,of,11
33,a,10
...,...,...
1,again.,1
40,On,1
122,"providers,",1
123,"Malik,",1


So we have a dataframe with word occurence frequency in the article. 

**But** there is punctuation messing this up. For example, we see that ```again.``` appears once, as does ```providers,```.  

One option to fix this would be to strip out the punctuation using Python string manipulation. But you could also use regular expressions to remove the punctuation. Below is a hacky example, but you can probably find a better solution. 

In [112]:
import re  ## imports the regular expression library
words_wo_punctuation = re.sub(r'[^\w\s]','',body.replace('<p>','').replace('</p>','')).split()  

Note that the regex ```r'[^\w\s]'``` substitutes anything in ```body``` that is not a word ```\w``` or and blank space ```\s``` with the empty string ```''```.

In [113]:
unique_words = list(set(words_wo_punctuation))
print(len(unique_words))
count_dictionary = {'word': unique_words, 'count': [words_wo_punctuation.count(w) for w in unique_words]}

225


In [117]:
df = pd.DataFrame(count_dictionary)
df.sort_values(by='count', ascending=False)

,word,count
135,the,16
62,to,14
20,Google,14
196,users,13
3,of,11
...,...,...
89,KnowBe4,1
90,result,1
91,4,1
92,resulted,1
